In [18]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 
from scipy.stats import truncnorm
import datetime

Stocks Setup
==============

In [81]:
#Subfolder where the file was saved
File_Location='C://Users/pmlef/Documents/python_work/Python_for_Data_Analysis/'
stocks=pd.read_csv(File_Location+'stocks-us-adjClose.csv',parse_dates=True,dayfirst=True,na_values=".")
#Rename first Row
stocks.rename(index=str, columns={"compiled from Yahoo! Finance data by Matt Borthwick": "Date"}, inplace=True)
#Set Date column
stocks['Date'] = pd.to_datetime(stocks['Date'], format = '%Y-%m-%d',  errors='coerce')

In [82]:
stocks.columns

Index(['Date', 'ED', 'DD', 'CVX', 'FL', 'CAT', 'IP', 'SJW', 'F', 'LLY',
       ...
       'EXTN', 'VYGR', 'ACG', 'MIME', 'TCRZ', 'MCX', 'EDIT', 'LMHA', 'UA',
       'BTU'],
      dtype='object', length=711)

In [83]:
stocks.head()

,Date,ED,DD,CVX,FL,CAT,IP,SJW,F,LLY,...,EXTN,VYGR,ACG,MIME,TCRZ,MCX,EDIT,LMHA,UA,BTU
0,1970-01-02,0.307997,0.000480,0.582503,1.857836,1.475225,1.808415,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1970-01-05,0.320831,0.000481,0.585290,1.839623,1.470783,1.873423,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1970-01-06,0.316553,0.000477,0.576929,1.845694,1.435234,1.855694,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1970-01-07,0.312275,0.000475,0.575535,1.809266,1.390800,1.814324,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970-01-08,0.312275,0.000469,0.586683,1.821408,1.395244,1.832054,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Create Useful Variables
----------------------------

In [86]:
stocks['year'], stocks['month'], stocks['day'], stocks['day_name'], stocks['decade'], stocks['quarter']=stocks['Date'].dt.year, stocks['Date'].dt.month, stocks['Date'].dt.day, stocks['Date'].dt.weekday_name, (stocks['Date'].dt.year // 10) * 10, stocks['Date'].dt.quarter
stocks['half_yr']=[2 if x>2 else 1 for x in stocks['Date'].dt.quarter]
stocks['avg_all']=stocks.loc[:,'ED':'BTU'].mean(axis=1)
stocks['sum_all']=stocks.loc[:,'ED':'BTU'].sum(axis=1)
stocks['count_all']=stocks.loc[:,'ED':'BTU'].count(axis=1)
stocks['min_all']=stocks.loc[:,'ED':'BTU'].min(axis=1)
stocks['max_all']=stocks.loc[:,'ED':'BTU'].max(axis=1)

In [87]:
stocks.head()

,Date,ED,DD,CVX,FL,CAT,IP,SJW,F,LLY,...,day,day_name,decade,quarter,half_yr,avg_all,sum_all,count_all,min_all,max_all
0,1970-01-02,0.307997,0.000480,0.582503,1.857836,1.475225,1.808415,NaN,NaN,NaN,...,2,Friday,1970,1,1,1.005409,6.032456,6,0.000480,1.857836
1,1970-01-05,0.320831,0.000481,0.585290,1.839623,1.470783,1.873423,NaN,NaN,NaN,...,5,Monday,1970,1,1,1.015072,6.090431,6,0.000481,1.873423
2,1970-01-06,0.316553,0.000477,0.576929,1.845694,1.435234,1.855694,NaN,NaN,NaN,...,6,Tuesday,1970,1,1,1.005097,6.030581,6,0.000477,1.855694
3,1970-01-07,0.312275,0.000475,0.575535,1.809266,1.390800,1.814324,NaN,NaN,NaN,...,7,Wednesday,1970,1,1,0.983779,5.902675,6,0.000475,1.814324
4,1970-01-08,0.312275,0.000469,0.586683,1.821408,1.395244,1.832054,NaN,NaN,NaN,...,8,Thursday,1970,1,1,0.991356,5.948133,6,0.000469,1.832054


Indicators Setup/Cleaning
====================

In [170]:
import csv

# remove warnings
import warnings
warnings.filterwarnings('ignore')

# I ran into errors importing the file because of non-ascii characters in the heading
indicators=[]
with open('indicators.csv', newline='', encoding='utf-8', errors='ignore') as f:
    reader = csv.reader(f)
    for row in reader:
        indicators.append(row)

indicators=pd.DataFrame(indicators)
indicators.columns=indicators.iloc[0]
Column_Reference=indicators.iloc[1].copy()
#Drop useless rows (last row is empty)
indicators.drop([0,1,579],inplace=True)

#Set Date in datetime format
indicators['DATE'] = pd.to_datetime(indicators['DATE'], format = '%m/%d/%Y',  errors='coerce')

#Deal with commas in DataFrame
PAYEMS=indicators['PAYEMS'].copy()
PAYEMS=PAYEMS.str.replace(',', '')
indicators['PAYEMS']=PAYEMS
#Set values to integers
indicators.loc[:,'PAYEMS':'IPMAN']=indicators.loc[:,'PAYEMS':'IPMAN'].convert_objects(convert_numeric=True)


In [171]:
indicators.head()

,DATE,PAYEMS,PAYNSA,AHETPI,USCONS,USCONS_NSA,MANEMP,MANEMP_NSA,USTRADE,HOUST,...,CPIAUCSL,UMCSENT,RRSFS,RSXFS,USSLIND,SPCS20RSA,SPCS2-RNSA,KCFSI,DGORDER,IPMAN
2,1970-01-01,71176,70229,3.31,3615,3279,18424,18254,7425.0,1085,...,37.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1970-02-01,71304,70332,3.33,3703,3306,18361,18205,7441.8,1305,...,38.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970-03-01,71452,70786,3.35,3697,3402,18360,18233,7465.1,1319,...,38.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1970-04-01,71348,71105,3.36,3669,3549,18207,18084,7447.2,1264,...,38.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1970-05-01,71123,71123,3.38,3634,3621,18029,17897,7453.1,1290,...,38.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
Column_Reference

0
DATE                                                       Desc
PAYEMS                                         Non Farm Payroll
PAYNSA                                     Non Farm Payroll NSA
AHETPI        Average Hourly Earnings of Production and Nons...
USCONS                              All Employees: Construction
USCONS_NSA                      All Employees: Construction NSA
MANEMP                             All Employees: Manufacturing
MANEMP_NSA                     All Employees: Manufacturing NSA
USTRADE                             All Employees: Retail Trade
HOUST         Housing Starts: Total: New Privately Owned Hou...
UNRATE                               Civilian Unemployment Rate
UNRATENSA                        Civilian Unemployment Rate NSA
EMRATIO                    Civilian Employment-Population Ratio
UEMPMEAN                       Average Duration of Unemployment
UEMPMED                         Median Duration of Unemployment
U2RATE                            Unem

Create Useful Variables
------------------------------

Setting time variables

In [173]:
indicators['year'], indicators['month'], indicators['day'], indicators['day_name'], indicators['decade'], indicators['quarter']=indicators['DATE'].dt.year, indicators['DATE'].dt.month, indicators['DATE'].dt.day, indicators['DATE'].dt.weekday_name, (indicators['DATE'].dt.year // 10) * 10, indicators['DATE'].dt.quarter
indicators['half_yr']=[2 if x>2 else 1 for x in indicators['DATE'].dt.quarter]

In [174]:
indicators.head()

,DATE,PAYEMS,PAYNSA,AHETPI,USCONS,USCONS_NSA,MANEMP,MANEMP_NSA,USTRADE,HOUST,...,KCFSI,DGORDER,IPMAN,year,month,day,day_name,decade,quarter,half_yr
2,1970-01-01,71176,70229,3.31,3615,3279,18424,18254,7425.0,1085,...,NaN,NaN,NaN,1970,1,1,Thursday,1970,1,1
3,1970-02-01,71304,70332,3.33,3703,3306,18361,18205,7441.8,1305,...,NaN,NaN,NaN,1970,2,1,Sunday,1970,1,1
4,1970-03-01,71452,70786,3.35,3697,3402,18360,18233,7465.1,1319,...,NaN,NaN,NaN,1970,3,1,Sunday,1970,1,1
5,1970-04-01,71348,71105,3.36,3669,3549,18207,18084,7447.2,1264,...,NaN,NaN,NaN,1970,4,1,Wednesday,1970,2,1
6,1970-05-01,71123,71123,3.38,3634,3621,18029,17897,7453.1,1290,...,NaN,NaN,NaN,1970,5,1,Friday,1970,2,1


Group Year by Month Stocks
------------------------

In [175]:
monthlyStocks = stocks.groupby([stocks['year'],stocks['month']]).sum()
monthlyStocks[:15]

ED        DD        CVX         FL        CAT         IP  \
year month                                                                   
1970 1      6.492181  0.009614  11.691859  37.226242  28.713581  36.121019   
     2      5.832151  0.008237   9.746552  31.922654  25.607614  32.125227   
     3      6.619700  0.009298  10.741828  35.266172  31.068611  35.460333   
     4      6.953335  0.010621  10.862128  35.949432  32.490515  37.836296   
     5      6.116030  0.010332   9.803456  29.858963  26.469638  31.637054   
     6      5.980004  0.011769  10.561248  32.313719  28.042621  33.775198   
     7      6.135837  0.012119  10.482059  33.295137  25.389878  35.769440   
     8      5.703864  0.011785  11.006244  33.557174  24.550067  35.242502   
     9      5.776629  0.011857  11.447580  35.473841  25.656488  36.736831   
     10     6.122633  0.012014  12.291917  37.294966  27.478301  37.506643   
     11     5.446119  0.011305  11.464414  33.932827  25.554288  31.804914   
     12     6.578032  0.013233  13.779552  40.681592  32.459407  37.915280   
1971 1      6.800681  0.012557  12.507021  38.695589  31.895086  35.183212   
     2      6.246504  0.012599  12.273411  41.987579  31.641804  34.802776   
     3      7.506880  0.015156  15.866479  60.015840  38.841303  42.643050   

            SJW   F  LLY  AVP    ...      BTU  day  decade  quarter  half_yr  \
year month                       ...                                           
1970 1      NaN NaN  NaN  NaN    ...      NaN  352   41370       21       21   
     2      NaN NaN  NaN  NaN    ...      NaN  267   37430       19       19   
     3      NaN NaN  NaN  NaN    ...      NaN  324   41370       21       21   
     4      NaN NaN  NaN  NaN    ...      NaN  345   43340       44       22   
     5      NaN NaN  NaN  NaN    ...      NaN  331   41370       42       21   
     6      NaN NaN  NaN  NaN    ...      NaN  329   43340       44       22   
     7      NaN NaN  NaN  NaN    ...      NaN  373   43340       66       44   
     8      NaN NaN  NaN  NaN    ...      NaN  341   41370       63       42   
     9      NaN NaN  NaN  NaN    ...      NaN  330   41370       63       42   
     10     NaN NaN  NaN  NaN    ...      NaN  353   43340       88       44   
     11     NaN NaN  NaN  NaN    ...      NaN  294   39400       80       40   
     12     NaN NaN  NaN  NaN    ...      NaN  343   43340       88       44   
1971 1      NaN NaN  NaN  NaN    ...      NaN  330   39400       20       20   
     2      NaN NaN  NaN  NaN    ...      NaN  255   37430       19       19   
     3      NaN NaN  NaN  NaN    ...      NaN  360   45310       23       23   

              avg_all     sum_all  count_all   min_all    max_all  
year month                                                         
1970 1      20.042416  120.254496        126  0.009614  37.314725  
     2      17.540406  105.242435        114  0.008237  32.586561  
     3      19.860990  119.165942        126  0.009298  35.605547  
     4      20.683721  124.102327        132  0.010621  37.837909  
     5      17.315912  103.895473        126  0.010332  31.645865  
     6      18.447426  110.684559        132  0.011769  33.972167  
     7      18.514078  111.084470        132  0.012119  35.769440  
     8      18.345273  110.071636        126  0.011785  35.242502  
     9      19.183871  115.103226        126  0.011857  36.736921  
     10     20.117746  120.706474        132  0.012014  37.888624  
     11     18.035644  108.213867        120  0.011305  33.950550  
     12     21.904516  131.427096        132  0.013233  40.681592  
1971 1      20.849024  125.094146        120  0.012557  38.695589  
     2      21.160779  126.964673        114  0.012599  41.987579  
     3      27.481451  164.888708        138  0.015156  60.015840  

[15 rows x 719 columns]

Group Year by Month Indicators
------------------------

In [193]:
monthlyIndicators = indicators.groupby([indicators['year'],indicators['month']]).sum()
#Keep only values that match stock time frame
monthlyIndicators=monthlyIndicators.iloc[:-10].copy()

In [327]:
monthlyIndicators[:15]

0           PAYEMS  PAYNSA  AHETPI  USCONS  USCONS_NSA  MANEMP  MANEMP_NSA  \
year month                                                                   
1970 1       71176   70229    3.31    3615        3279   18424       18254   
     2       71304   70332    3.33    3703        3306   18361       18205   
     3       71452   70786    3.35    3697        3402   18360       18233   
     4       71348   71105    3.36    3669        3549   18207       18084   
     5       71123   71123    3.38    3634        3621   18029       17897   
     6       71029   71766    3.39    3636        3803   17930       18063   
     7       71053   71000    3.41    3645        3888   17877       17788   
     8       70933   70900    3.43    3649        3940   17779       17889   
     9       70948   71257    3.45    3618        3847   17692       17941   
     10      70519   71022    3.46    3626        3836   17173       17341   
     11      70409   70983    3.47    3643        3758   17024       17148   
     12      70790   71564    3.50    3688        3613   17309       17336   
1971 1       70866   69922    3.52    3643        3296   17280       17100   
     2       70806   69844    3.54    3633        3228   17216       17050   
     3       70859   70209    3.56    3674        3377   17154       17017   

0           USTRADE  HOUST  UNRATE   ...     USSLIND  SPCS20RSA  SPCS2-RNSA  \
year month                           ...                                      
1970 1       7425.0   1085     3.9   ...         NaN        NaN         NaN   
     2       7441.8   1305     4.2   ...         NaN        NaN         NaN   
     3       7465.1   1319     4.4   ...         NaN        NaN         NaN   
     4       7447.2   1264     4.6   ...         NaN        NaN         NaN   
     5       7453.1   1290     4.8   ...         NaN        NaN         NaN   
     6       7453.6   1385     4.9   ...         NaN        NaN         NaN   
     7       7462.9   1517     5.0   ...         NaN        NaN         NaN   
     8       7460.5   1399     5.1   ...         NaN        NaN         NaN   
     9       7478.9   1534     5.4   ...         NaN        NaN         NaN   
     10      7495.7   1580     5.5   ...         NaN        NaN         NaN   
     11      7466.6   1647     5.9   ...         NaN        NaN         NaN   
     12      7514.0   1893     6.1   ...         NaN        NaN         NaN   
1971 1       7556.4   1828     5.9   ...         NaN        NaN         NaN   
     2       7554.9   1741     5.9   ...         NaN        NaN         NaN   
     3       7569.4   1910     6.0   ...         NaN        NaN         NaN   

0           KCFSI  DGORDER  IPMAN  day  decade  quarter  half_yr  
year month                                                        
1970 1        NaN      NaN    NaN    1    1970        1        1  
     2        NaN      NaN    NaN    1    1970        1        1  
     3        NaN      NaN    NaN    1    1970        1        1  
     4        NaN      NaN    NaN    1    1970        2        1  
     5        NaN      NaN    NaN    1    1970        2        1  
     6        NaN      NaN    NaN    1    1970        2        1  
     7        NaN      NaN    NaN    1    1970        3        2  
     8        NaN      NaN    NaN    1    1970        3        2  
     9        NaN      NaN    NaN    1    1970        3        2  
     10       NaN      NaN    NaN    1    1970        4        2  
     11       NaN      NaN    NaN    1    1970        4        2  
     12       NaN      NaN    NaN    1    1970        4        2  
1971 1        NaN      NaN    NaN    1    1970        1        1  
     2        NaN      NaN    NaN    1    1970        1        1  
     3        NaN      NaN    NaN    1    1970        1        1  

[15 rows x 30 columns]

In [197]:
monthlyIndicators.tail()

0           PAYEMS  PAYNSA  AHETPI  USCONS  USCONS_NSA  MANEMP  MANEMP_NSA  \
year month                                                                   
2016 11     145257  146482   21.72    6811        6903   12339       12338   
     12     145437  146270   21.78    6822        6700   12351       12354   
2017 1      145696  143393   21.81    6873        6459   12369       12277   
     2      145896  144423   21.85    6919        6527   12390       12315   
     3      145969  145078   21.89    6922        6634   12400       12340   

0           USTRADE  HOUST  UNRATE   ...     USSLIND   SPCS20RSA  SPCS2-RNSA  \
year month                           ...                                       
2016 11     15875.4   1149     4.6   ...        1.72  192.280166  191.902941   
     12     15890.4   1268     4.7   ...        1.42  193.672822  192.338784   
2017 1      15912.8   1236     4.8   ...        1.53  195.220236  192.756178   
     2      15891.2   1288     4.7   ...        1.36  196.438976  193.522829   
     3      15859.4   1189     4.5   ...        1.50  198.375126  195.397876   

0           KCFSI   DGORDER     IPMAN  day  decade  quarter  half_yr  
year month                                                            
2016 11     -0.20  223079.0  103.1081    1    2010        4        2  
     12     -0.43  223681.0  103.3225    1    2010        4        2  
2017 1      -0.34  224395.0  103.7558    1    2010        1        1  
     2      -0.56  227461.0  104.0198    1    2010        1        1  
     3      -0.55  232817.0  103.3405    1    2010        1        1  

[5 rows x 30 columns]

Append Indicators to Stock Data
-------------------------------------

In [320]:
#Only keep the same number of rows that exist in the indicators dataset
FullData=monthlyStocks[:-6].copy()
#Concatenate Stocks and Indicators
FullData=pd.concat([FullData, monthlyIndicators], axis=1)

In [348]:
FullData.index.names

FrozenList(['year', 'month'])

In [356]:
FullData.columns

Index(['ED', 'DD', 'CVX', 'FL', 'CAT', 'IP', 'SJW', 'F', 'LLY', 'AVP',
       ...
       'USSLIND', 'SPCS20RSA', 'SPCS2-RNSA', 'KCFSI', 'DGORDER', 'IPMAN',
       'day', 'decade', 'quarter', 'half_yr'],
      dtype='object', length=749)

In [357]:
FullData[:15]

ED        DD        CVX         FL        CAT         IP  \
year month                                                                   
1970 1      6.492181  0.009614  11.691859  37.226242  28.713581  36.121019   
     2      5.832151  0.008237   9.746552  31.922654  25.607614  32.125227   
     3      6.619700  0.009298  10.741828  35.266172  31.068611  35.460333   
     4      6.953335  0.010621  10.862128  35.949432  32.490515  37.836296   
     5      6.116030  0.010332   9.803456  29.858963  26.469638  31.637054   
     6      5.980004  0.011769  10.561248  32.313719  28.042621  33.775198   
     7      6.135837  0.012119  10.482059  33.295137  25.389878  35.769440   
     8      5.703864  0.011785  11.006244  33.557174  24.550067  35.242502   
     9      5.776629  0.011857  11.447580  35.473841  25.656488  36.736831   
     10     6.122633  0.012014  12.291917  37.294966  27.478301  37.506643   
     11     5.446119  0.011305  11.464414  33.932827  25.554288  31.804914   
     12     6.578032  0.013233  13.779552  40.681592  32.459407  37.915280   
1971 1      6.800681  0.012557  12.507021  38.695589  31.895086  35.183212   
     2      6.246504  0.012599  12.273411  41.987579  31.641804  34.802776   
     3      7.506880  0.015156  15.866479  60.015840  38.841303  42.643050   

            SJW   F  LLY  AVP   ...     USSLIND  SPCS20RSA  SPCS2-RNSA  KCFSI  \
year month                      ...                                             
1970 1      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     2      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     3      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     4      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     5      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     6      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     7      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     8      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     9      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     10     NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     11     NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     12     NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
1971 1      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     2      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   
     3      NaN NaN  NaN  NaN   ...         NaN        NaN         NaN    NaN   

            DGORDER  IPMAN  day  decade  quarter  half_yr  
year month                                                 
1970 1          NaN    NaN    1    1970        1        1  
     2          NaN    NaN    1    1970        1        1  
     3          NaN    NaN    1    1970        1        1  
     4          NaN    NaN    1    1970        2        1  
     5          NaN    NaN    1    1970        2        1  
     6          NaN    NaN    1    1970        2        1  
     7          NaN    NaN    1    1970        3        2  
     8          NaN    NaN    1    1970        3        2  
     9          NaN    NaN    1    1970        3        2  
     10         NaN    NaN    1    1970        4        2  
     11         NaN    NaN    1    1970        4        2  
     12         NaN    NaN    1    1970        4        2  
1971 1          NaN    NaN    1    1970        1        1  
     2          NaN    NaN    1    1970        1        1  
     3          NaN    NaN    1    1970        1        1  

[15 rows x 749 columns]